<a href="https://colab.research.google.com/github/smartgrids-aau/GreenCodesPython/blob/main/Exemple_3_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Example 3.6

import numpy as np
import pandas as pd
from scipy.optimize import root
import scipy.io as sio

# Timing start
import time
t_start = time.process_time()

# Load Excel data
file_path = 'PV Modeling Book Data Source.xls'

df = pd.read_excel(file_path, sheet_name='Source 6', skiprows=9, nrows=4380)
G = df.iloc[:, 8].to_numpy()     # Column I: Solar radiation
Tc = df.iloc[:, 9].to_numpy()    # Column J: Cell temperature
Vm = df.iloc[:, 11].to_numpy()   # Column L: Module voltage
Im = df.iloc[:, 12].to_numpy()   # Column M: Module current


sio.savemat('input_variable.mat', {'G': G, 'Tc': Tc, 'Vm': Vm, 'Im': Im})

# Constants and Configuration
h1 = 20
h2 = 0.1444
Ns = 5
Np = 4
Cn = 50
Am = 0.9291
Ra = 0.8
Km = 0.175
Rou = 1000
g = 9.81
d1 = 33.5e-3
d2 = 160e-3
beta1 = np.deg2rad(38)
beta2 = np.deg2rad(33)
b1 = 5.4e-3
b2 = 2.2e-3

# PV array calculations
Va = Ns * Vm
Ia = Np * Im
Pao = Va * Ia
A = Ns * Np * Am
Pai = A * G
effa = np.divide(Pao, Pai, out=np.zeros_like(Pao), where=Pai != 0)

# Save PV array variables
sio.savemat('output_variable_PV_array.mat', {'Ia': Ia, 'Va': Va, 'Pao': Pao, 'Pai': Pai, 'effa': effa, 'A': A})

# Motor calculations
Va = 0.95 * Va
Ia = 0.9 * Ia
Ebb = Va - (Ra * Ia)
Eb = np.where(Ebb >= 0, Ebb, 0)
Ia = np.where(Ebb >= 0, Ia, 0)
Va = np.where(Ebb >= 0, Va, 0)
Tm = Km * Ia
Tm1 = Tm + (Tm == 0).astype(float)
Kp = Rou * 2 * np.pi * b1 * (d1 / 2) ** 2 * np.tan(beta1) * ((d2 / 2) ** 2 - ((b1 * (d1 / 2) ** 2 * np.tan(beta1)) / (b2 * np.tan(beta2))))
Pdev = Eb * Ia
Omega = np.abs(np.sqrt(np.divide(Km * Ia, Kp, out=np.zeros_like(Ia), where=Kp != 0)))
Pmo = Pdev
Pmi = Pao * 0.9
PMI2 = Pmi + (Pmi == 0).astype(float)
effm = np.divide(Pmo, PMI2, out=np.zeros_like(Pmo), where=PMI2 != 0)

# Save motor variables
sio.savemat('output_variable_Motor.mat', {'Ia': Ia, 'Va': Va, 'Pmo': Pmo, 'Pmi': Pmi, 'effm': effm, 'Tm': Tm, 'Omega': Omega})

# Pump calculations
Tp = Tm
Eh = Tp * Omega
Ppo = np.where(Ebb >= 0, Eh, 0)
Ppi = Pmo
PPI2 = Ppi + (Ppi == 0).astype(float)
effpp = np.divide(Ppo, PPI2, out=np.zeros_like(Ppo), where=PPI2 != 0)
effp = np.where(effpp <= 0.95, effpp, 0)

QQ = np.zeros_like(Eh)
for ii in range(len(Eh)):
    coeffs = [h2 * 2.725, 0, h1 * 2.725, -Eh[ii]]
    roots_result = np.roots(coeffs)
    real_roots = [r.real for r in roots_result if np.isreal(r) and r.real > 0]
    QQ[ii] = real_roots[0] if real_roots else 0

Q2 = QQ + (QQ == 0).astype(float)
H = np.divide(Eh, 2.725 * Q2, out=np.zeros_like(Eh), where=Q2 != 0)

# System efficiency
effsub = effm * effp
effoverall = effa * effm * effp
QQ = np.where((Ebb >= 0) & (effpp <= 0.95), QQ, 0)
Q = np.insert(QQ, 0, 0)

# Demand and storage simulation
d = 2.5
Cr = np.zeros_like(Q)
Qexcess_pv = np.zeros_like(Q)
Qexcess_s = np.zeros_like(Q)
SOC = np.zeros_like(Q)
Qdef_pv = np.zeros_like(Q)
Qdeficit_s = np.zeros_like(Q)
X = Q[1:] - d

Qdef_pv[1:] = np.where(X < 0, np.abs(X), 0)
Qexcess_pv[1:] = np.where(X >= 0, np.abs(X), 0)

for i in range(len(Q) - 1):
    Cr[i+1] = max(Cr[i] + X[i], 0)
    SOC[i+1] = min(Cr[i+1] / Cn, 1)
    if SOC[i+1] >= 1:
        Qexcess_s[i+1] = Cr[i+1] - Cn
        Cr[i+1] = Cn
    else:
        Qexcess_s[i+1] = 0
    Qdeficit_s[i+1] = max(-(Cr[i] + X[i]), 0) if (Cr[i] + X[i]) < 0 else 0

Q = Q[1:]
Qexcess_pv = Qexcess_pv[1:]
Qexcess_s = Qexcess_s[1:]
Qdeficit_s = Qdeficit_s[1:]
Qdef_pv = Qdef_pv[1:]
Cres = Cr[1:]
SOC = SOC[1:]
D = np.full_like(Q, d)
LLPh = Qdeficit_s / D
LLP = np.sum(Qdeficit_s) / np.sum(D)

# Save final system variables
sio.savemat('output_variable_System.mat', {
    'Q': Q, 'H': H, 'D': D, 'Qdef_pv': Qdef_pv,
    'Qdeficit_s': Qdeficit_s, 'Qexcess_pv': Qexcess_pv,
    'Qexcess_s': Qexcess_s, 'Cr': Cres, 'SOC': SOC,
    'effoverall': effoverall, 'effsub': effsub,
    'LLPh': LLPh, 'LLP': LLP
})


t_end = time.process_time()
execution_time = t_end - t_start
execution_time


1.48055087